Calculating tf-idf stats
=============

In [1]:
import json
import nltk
import operator
import os

from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer

/usr/local/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


Load vocab...

In [2]:
vocab_fn = "vocab.txt"
with open(vocab_fn, "r+") as f:
    vocab = json.load(f)
vocab_set = {x for v in vocab.values() for x in v} # unpacking this a little...

Load recorded counts (i.e. the tf parts of tf-idf) and find some interesting files.

In [3]:
counts_per_file_fn = "counts_per_file.txt"
with open(counts_per_file_fn, "r+") as f:
    counts_per_file = json.load(f)

counts_agg_fn = "counts_aggregate.txt"
with open(counts_agg_fn, "r+") as f:
    counts_agg = json.load(f)

sorted(counts_agg.items(), key=lambda x: -x[1])[:5]

[('water', 579), ('power', 475), ('road', 374), ('port', 255), ('roads', 199)]

In [4]:
# most frequent occurrences of the specific word 'water'
counts_water = {}
for file, vals in counts_per_file.items():
    if 'water' in vals and vals['water'] > 0:
        counts_water[file] = vals['water']

sorted(counts_water.items(), key=lambda x: -x[1])[:5]

[('article-96.cfm_nzh.txt', 22),
 ('article-300.cfm_nzh.txt', 22),
 ('article-112.cfm_nzh.txt', 17),
 ('article-163.cfm_nzh.txt', 16),
 ('article-477.cfm_nzh.txt', 15)]

Load words...

In [5]:
article_dir = "../../TextData/CanterburyNewsStoriesCleaned/"

words = {}
for article_fn in os.listdir(article_dir):
    with open(os.path.join(article_dir, article_fn), "r+") as f:
        raw = f.read()
        
    words[article_fn] = raw

The actual interesting parts!

In [6]:
tfidf = TfidfVectorizer(stop_words='english')
tfs = tfidf.fit_transform(words.values())

feature_names = tfidf.get_feature_names()

In [7]:
# looking at a specific doc (one of the ones from above)...
response = tfidf.transform([words['article-300.cfm_nzh.txt']])
for col in response.nonzero()[1]:
    if feature_names[col] in vocab_set:
        print (feature_names[col], ' - ', response[0, col])

water  -  0.128132393315
truck  -  0.00968514466019
traffic  -  0.0470297248563
sewers  -  0.0106847034851
sewerage  -  0.00991400942576
sewer  -  0.0115344954137
sewage  -  0.00999620995252
sanitation  -  0.011916613096
roads  -  0.0203934984642
road  -  0.134730017921
power  -  0.0761649531149
port  -  0.0274129313005
fuel  -  0.0437262337802
energy  -  0.0077503370286
electricity  -  0.00890889869092
drinking  -  0.0286361096741
distribution  -  0.0104638841988
cars  -  0.00876430034165
car  -  0.0420513097884
bus  -  0.0154436433876
airport  -  0.00753239073078


In [8]:
# poking around w/inverse document frequency...

idf = tfidf.idf_

sorted_idf_list = sorted(zip(feature_names, idf), key=lambda x: -x[1])
[item for item in sorted_idf_list if item[0] in vocab_set][:10]

[('commutes', 7.2952660014396464),
 ('petroleum', 7.2952660014396464),
 ('potable', 7.2952660014396464),
 ('sanitary', 6.8898008933314818),
 ('commute', 6.602118820879701),
 ('electric', 6.602118820879701),
 ('planes', 6.3789752695654913),
 ('commuter', 6.1966537127715364),
 ('utility', 6.1966537127715364),
 ('airports', 6.0425030329442784)]